In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
data = pd.read_csv("Train.csv")
data.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [25]:
data = data.drop(columns='ID')

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 630.4+ KB


In [27]:
data.isnull().values.sum()

1582

In [28]:
data.isnull().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [29]:
cat_data = data.select_dtypes(include=['object']).copy()
cat_data.insert(1,'Family_Size', data['Family_Size'], True)
print(cat_data.isnull().sum())

Gender              0
Family_Size       335
Ever_Married      140
Graduated          78
Profession        124
Spending_Score      0
Var_1              76
Segmentation        0
dtype: int64


In [30]:
nulls = ['Ever_Married','Profession', 'Graduated', 'Var_1', 'Family_Size']
for feature in nulls:
    cat_data = cat_data.fillna(cat_data[feature].value_counts().index[0])

In [31]:
cat_data.isnull().sum()

Gender            0
Family_Size       0
Ever_Married      0
Graduated         0
Profession        0
Spending_Score    0
Var_1             0
Segmentation      0
dtype: int64

In [32]:
categoricals = cat_data.columns.values
np.append(categoricals, ['Family_Size'])
for categorical in categoricals:
    data = data.drop(columns=categorical)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              8068 non-null   int64  
 1   Work_Experience  7239 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 126.2 KB


In [34]:
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Gender          8068 non-null   object
 1   Family_Size     8068 non-null   object
 2   Ever_Married    8068 non-null   object
 3   Graduated       8068 non-null   object
 4   Profession      8068 non-null   object
 5   Spending_Score  8068 non-null   object
 6   Var_1           8068 non-null   object
 7   Segmentation    8068 non-null   object
dtypes: object(8)
memory usage: 504.4+ KB


In [35]:
for categorical in categoricals:
    data.insert(1, categorical, cat_data[categorical], True)

In [45]:
def draw_bar_plot(feature, cat_data):
    feature_count = cat_data[feature].value_counts()
    guys = feature_count.index
    vals = feature_count.values
    plt.barh(guys,vals)
    plt.title(feature)
    plt.tight_layout()
    plt.show()
    
def draw_pie(feature, cat_data):
    feature_count = cat_data[feature].value_counts()
    guys = feature_count.index
    vals = feature_count.values
    plt.pie(vals, labels=guys, wedgeprops = {'edgecolor' : 'black'}, shadow=True, startangle=90)
    plt.title(feature)
    plt.tight_layout()
    plt.show()

In [ ]:
features = data.columns.values
for feature in features:
    if feature == 'Family_Size':
        continue
    draw_bar_plot(feature, data)
    draw_pie(feature, data)

In [36]:
data.isnull().sum()

Age                  0
Segmentation         0
Var_1                0
Spending_Score       0
Profession           0
Graduated            0
Ever_Married         0
Family_Size          0
Gender               0
Work_Experience    829
dtype: int64

In [37]:
from sklearn.impute import KNNImputer
cat_dt = data.select_dtypes(include=['object']).copy()
cat_dummies = pd.get_dummies(cat_dt, drop_first=True)

In [ ]:
for categorical in cat_dt.columns.values:
    data = data.drop(columns=categorical)
for categorical in cat_dummies.columns.values:
    data.insert(1, categorical, cat_dummies[categorical], True)

In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

In [51]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
data = pd.DataFrame(imputer.fit_transform(data),columns = data.columns)

In [53]:
data.isnull().values.sum()

0